In [11]:
from wmf import wmf 
import argparse
import textwrap
import os 
import alarmas as al
import pandas as pd 
import datetime as dt 

In [39]:
def model_update_norain_last(key, ruta_rain_temp, DictUpdate, ruta_bck_sto, ruta_sto, DictStore, date, umbral = 1):
    #lee la lluvia temporal
    rainHist = pd.read_csv(ruta_rain_temp, 
        index_col=3, parse_dates=True, 
        infer_datetime_format=True, header=None)
    #Suma la lluvia
    if rainHist[2].sum() < umbral:
        if DictUpdate['-t '+k[3:]]['Nombre'] <> 'None':
            #Remplaza el archivo
            comando = 'cp '+ruta_bck_sto+DictUpdate['-t '+key[3:]]['Nombre']+' '+ruta_sto+DictStore[key]['Nombre']
            os.system(comando)
            #Actualiza la fecha de actualizacion 
            DictUpdate['-t '+key[3:]]['LastUpdate'] = date
            return 0
        else:
            return 1
    else:
        return 1

def model_update_norain_next(key, Rain, DictUpdate,ruta_bck_sto, ruta_sto, DictStore, date, umbral = 1):
    #Suma la lluvia
    if Rain.sum()<umbral:
        if DictUpdate['-t '+k[3:]]['Nombre'] <> 'None':
            #remplaza
            comando = 'cp '+ruta_bck_sto+DictUpdate['-t '+key[3:]]['Nombre']+' '+ruta_sto+DictStore[key]['Nombre']
            os.system(comando)
            #Actualiza la fecha de actualizacion 
            DictUpdate['-t '+key[3:]]['LastUpdate'] = date
            return 0
        else:
            return 1
    else:
        return 1

def model_update_norain(key, ruta_rain_temp, Rain, DictUpdate, ruta_bck_sto,ruta_sto,DictStore, date, umbral = 1):
    #lee la lluvia temporal
    rainHist = pd.read_csv(ruta_rain_temp, 
        index_col=3, parse_dates=True, 
        infer_datetime_format=True, header=None,)
    #Suma la lluvia
    if rainHist[2].sum() < umbral and Rain.sum()<umbral:
        if DictUpdate['-t '+k[3:]]['Nombre'] <> 'None':
            #Remplaza
            comando = 'cp '+ruta_bck_sto+DictUpdate['-t '+key[3:]]['Nombre']+' '+ruta_sto+DictStore[key]['Nombre']
            os.system(comando)
            #Actualiza la fecha de actualizacion 
            DictUpdate['-t '+key[3:]]['LastUpdate'] = date
            return 0
        else:
            return 1
    else:
        return 1

In [13]:
def Model_Update_Store(date,rutaRain,ruta_rain_hist,ruta_rain_temp,ruta_sto,ruta_bck_sto,DeltaT,DictStore,DictUpdate,ruta_configuracion_1):
    #informacion de la lluvia
    rain_bin, rain_hdr = wmf.__Add_hdr_bin_2route__(rutaRain)
    DataRain = wmf.read_rain_struct(rain_hdr)
    Rain = wmf.read_mean_rain(rain_hdr)

    ############################ ACTUALIZACION ###########################################

    #Fecha Actual pasada como parametro
    DateNow = pd.to_datetime(date)
    #Calcula la cantidad de horas desde la ultima actualizacion
    for k in DictUpdate.keys():
        dat = pd.to_datetime(DictUpdate[k]['LastUpdate'])
        deltaT = DateNow - dat
        DictUpdate[k].update({'Horas': deltaT.total_seconds()/3600.0})
    #si la cantidad de horas es inferior al umbral evalua si cumple la 
    #regla establecida 
    for k in DictStore.keys():
        #Evalua si esas condiciones se van a actualizar o no, y si cumplen
        #el tiempo sin ser actualizadas
        if DictStore[k]['Actualizar'] == 'True':
            if DictUpdate['-t '+k[3:]]['Horas'] > DictStore[k]['Tiempo']:

                #CASO 1: NO RAIN: no hay lluvia atras ni adelante.
                if DictStore[k]['Condition'][:-3] == 'No Rain':
                    #Obtiene las horas de la condicion
                    hours = float(DictStore[k]['Condition'][-3:-1])
                    al.get_rain_last_hours(ruta_rain_hist,ruta_rain_temp,hours, DeltaT)
                    estado = model_update_norain(k,ruta_rain_temp, Rain, DictUpdate, ruta_bck_sto,ruta_sto,DictStore, date, umbral)

                #CASO 2: NO RAIN NEXT: No hay lluvia adelante.
                elif DictStore[k]['Condition'][:-3] == 'No Rain Next':
                    estado = model_update_norain_next(k, Rain, DictUpdate,ruta_bck_sto, ruta_sto, DictStore, date, umbral)

                #CASO 3: NO RAIN LAST: no hay lluvia atras.
                elif DictStore[k]['Condition'][:-3] == 'No Rain Last':
                    #Obtiene las horas de la condicion
                    hours = float(DictStore[k]['Condition'][-3:-1])
                    al.get_rain_last_hours(ruta_rain_hist,ruta_rain_temp,hours, DeltaT)
                    estado = model_update_norain_last(k, ruta_rain_temp, DictUpdate, ruta_bck_sto, ruta_sto, DictStore, date, umbral)
                #CASO 4: ...

                #si esta diciendo lo que hace dice:
                if verbose:
                    if estado == 0:
                        print 'Aviso: Se han remplazado los estados de: '+k
                    else:
                        print 'Aviso: No se han remplazado los estados de: '+k

    #Actualiza las fechas dentro del archivo de configuracion 
    #Lee el archivo de configuracion
    ListConfig = al.get_rutesList(rutaConfig)
    #Obtiene las posiciones en la tabla
    pos = []
    key = '-t'
    for c,i in enumerate(ListConfig):
        if i.startswith('|'+key) or i.startswith('| '+key):
            pos.append(c)
    #Ordena las reglas 
    Keys = DictUpdate.keys()
    Keys.sort()
    #Obtiene las nuevas 
    for c,p in enumerate(pos):
        ListConfig[p] = '| '+Keys[c]+'|'+DictUpdate[Keys[c]]['Nombre']+'|'+DictUpdate[Keys[c]]['LastUpdate']+'|\n'
    #Escribe el nuevo archivo 
    f = open(rutaConfig,'w')
    f.writelines(ListConfig)
    f.close()

In [30]:
ruta_configuracion_1 = '/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/Op_AMVA60m/configfile.md'
RutasList = al.get_rutesList(ruta_configuracion_1)
ruta_out_rain = al.get_ruta(RutasList, 'ruta_rain')
lluvia_actual = ruta_out_rain + 'Lluvia_actual'

In [31]:
date='2018-01-15 21:50:00'
rutaRain=lluvia_actual+'.bin'
ruta_rain_hist = al.get_ruta(RutasList, 'ruta_rainHistoryFile')
ruta_sto = al.get_ruta(RutasList, 'ruta_almsim')
ruta_bck_sto = al.get_ruta(RutasList, 'ruta_bkc_alm')
DeltaT = float(al.get_ruta(RutasList, 'Dt[seg]'))
#Lista de calibraciones
DictStore = al.get_modelConfig_lines(RutasList, '-s', 'Store')
DictUpdate = al.get_modelConfig_lines(RutasList, '-t', 'Update')
ruta_rain_temp = '/var/tmp/RainTemp.hdr'
verbose=True
# al.Model_Update_Store(date,rutaRain,ruta_rain_hist,ruta_sto,ruta_bck_sto,DeltaT,DictStore,DictUpdate)

In [27]:
Model_Update_Store(date,rutaRain,ruta_rain_hist,ruta_rain_temp,ruta_sto,ruta_bck_sto,DeltaT,DictStore,DictUpdate,ruta_configuracion_1)

In [21]:
rutaConfig=ruta_configuracion_1

In [40]:
rain_bin, rain_hdr = wmf.__Add_hdr_bin_2route__(rutaRain)
DataRain = wmf.read_rain_struct(rain_hdr)
Rain = wmf.read_mean_rain(rain_hdr)

############################ ACTUALIZACION ###########################################

#Fecha Actual pasada como parametro
DateNow = pd.to_datetime(date)
#Calcula la cantidad de horas desde la ultima actualizacion
for k in DictUpdate.keys():
    dat = pd.to_datetime(DictUpdate[k]['LastUpdate'])
    deltaT = DateNow - dat
    DictUpdate[k].update({'Horas': deltaT.total_seconds()/3600.0})
#si la cantidad de horas es inferior al umbral evalua si cumple la 
#regla establecida 
for k in DictStore.keys():
    #Evalua si esas condiciones se van a actualizar o no, y si cumplen
    #el tiempo sin ser actualizadas
    if DictStore[k]['Actualizar'] == 'True':
        if DictUpdate['-t '+k[3:]]['Horas'] >= DictStore[k]['Tiempo']:

            #CASO 1: NO RAIN: no hay lluvia atras ni adelante.
            if DictStore[k]['Condition'][:-3] == 'No Rain':
                print 'this way'
                #Obtiene las horas de la condicion
                hours = float(DictStore[k]['Condition'][-3:-1])
                al.get_rain_last_hours(ruta_rain_hist,ruta_rain_temp,hours, DeltaT)
                estado = model_update_norain(k,ruta_rain_temp, Rain, DictUpdate, ruta_bck_sto,ruta_sto,DictStore, date)

            #CASO 2: NO RAIN NEXT: No hay lluvia adelante.
            elif DictStore[k]['Condition'][:-3] == 'No Rain Next':
                estado = model_update_norain_next(k, Rain, DictUpdate,ruta_bck_sto, ruta_sto, DictStore, date)

            #CASO 3: NO RAIN LAST: no hay lluvia atras.
            elif DictStore[k]['Condition'][:-3] == 'No Rain Last':
                #Obtiene las horas de la condicion
                hours = float(DictStore[k]['Condition'][-3:-1])
                al.get_rain_last_hours(ruta_rain_hist,ruta_rain_temp,hours, DeltaT)
                estado = model_update_norain_last(k, ruta_rain_temp, DictUpdate, ruta_bck_sto, ruta_sto, DictStore, date)
            #CASO 4: ...

            #si esta diciendo lo que hace dice:
            if verbose:
                if estado == 0:
                    print 'Aviso: Se han remplazado los estados de: '+k
                else:
                    print 'Aviso: No se han remplazado los estados de: '+k

#Actualiza las fechas dentro del archivo de configuracion 
#Lee el archivo de configuracion
ListConfig = al.get_rutesList(rutaConfig)
#Obtiene las posiciones en la tabla
pos = []
key = '-t'
for c,i in enumerate(ListConfig):
    if i.startswith('|'+key) or i.startswith('| '+key):
        pos.append(c)
#Ordena las reglas 
Keys = DictUpdate.keys()
Keys.sort()
#Obtiene las nuevas 
for c,p in enumerate(pos):
    ListConfig[p] = '| '+Keys[c]+'|'+DictUpdate[Keys[c]]['Nombre']+'|'+DictUpdate[Keys[c]]['LastUpdate']+'|\n'
#Escribe el nuevo archivo 
f = open(rutaConfig,'w')
f.writelines(ListConfig)
f.close()

this way


EmptyDataError: No columns to parse from file

In [26]:
 DictStore[k]['Condition'][:-3]

'No Rain'

In [37]:
# DictUpdate['-t '+k[3:]]['Horas'] #> 
DictStore[k]['Tiempo']

0.0

In [25]:
model_update_norain(k,ruta_rain_temp, Rain, DictUpdate, ruta_bck_sto,ruta_sto,DictStore, date)

0